In [24]:
using BenchmarkTools

In [131]:
module Ifc

using ComputedFieldTypes
using FunctionWrappers: FunctionWrapper

# Using an abstract parent type isn't necessary for this to work, 
# but I suspect it's something we'll often want to do
abstract type A end

struct B <: A
    x::Int
end

struct C <: A
    x::Float64
end

foo(a::A, y) = a.x + y

# This is the syntax I want to use eventually:
#=
@interface Foo{T} self::A begin
    foo(y::T)::Base.promote_op(+, Float64, T) = self.x + y
end
=#

# And the hypothetical `@interface` macro will produce something
# like the following:
@computed struct Foo{T}
    self::A
    foo::FunctionWrapper{Base.promote_op(+, Float64, T), Tuple{T}}

    Foo{T}(self) where {T} = new(self, y -> self.x + y)
end
foo(f::Foo{T}, y::T) where {T} = f.foo(y)

end

Ifc

In [140]:
# Demo
b = Ifc.B(1)
c = Ifc.C(2.0)
x_abstract = [b, c]

2-element Array{Ifc.A,1}:
 Ifc.B(1)  
 Ifc.C(2.0)

In [141]:
# The vector x_abstract has non-concrete type, so broadcast is slow:
y = zeros(2)
@benchmark $y .= Ifc.foo.($x_abstract, 2.0)

BenchmarkTools.Trial: 
  memory estimate:  80 bytes
  allocs estimate:  5
  --------------
  minimum time:     643.712 ns (0.00% GC)
  median time:      668.794 ns (0.00% GC)
  mean time:        766.344 ns (0.86% GC)
  maximum time:     17.130 μs (90.36% GC)
  --------------
  samples:          10000
  evals/sample:     163

In [142]:
# Instead, we can create an array of Foo interfaces:
x_interface = Ifc.Foo{Float64}.([b, c])

2-element Array{Ifc.Foo{Float64,Float64},1}:
 Ifc.Foo{Float64,Float64}(Ifc.B(1), FunctionWrappers.FunctionWrapper{Float64,Tuple{Float64}}(Ptr{Void} @0x000000012df313a0, Ptr{Void} @0x0000000128bbf710, Base.RefValue{Ifc.##1#2{Ifc.B}}(Ifc.#1), Ifc.##1#2{Ifc.B}))  
 Ifc.Foo{Float64,Float64}(Ifc.C(2.0), FunctionWrappers.FunctionWrapper{Float64,Tuple{Float64}}(Ptr{Void} @0x000000012df317c0, Ptr{Void} @0x0000000128bbf730, Base.RefValue{Ifc.##1#2{Ifc.C}}(Ifc.#1), Ifc.##1#2{Ifc.C}))

In [143]:
# Calling the interface's methods is much faster:
y = zeros(2)
@benchmark $y .= Ifc.foo.($x_interface, 2.0)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     32.700 ns (0.00% GC)
  median time:      33.698 ns (0.00% GC)
  mean time:        36.442 ns (0.00% GC)
  maximum time:     314.884 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     993

In [145]:
# Note that there's still some unavoidable overhead,
# probably due to the lack of inlining, so converting
# x to a vector of floats will still be faster:
y = zeros(2)
x_float = [1.0, 2.0]
@benchmark $y .= (+).($x_float, 2.0)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     10.867 ns (0.00% GC)
  median time:      11.178 ns (0.00% GC)
  mean time:        11.467 ns (0.00% GC)
  maximum time:     66.787 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999